In [1]:
# SET UP
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for i in range(99):
    model.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))
# Build a DNN with 20 hidden layers of 100 neurons each
# Use He initialization and the ELU activation function  

In [4]:
# Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset
model.add(keras.layers.Dense(10, activation='softmax')) # multiclass

In [5]:
# load dataset
(X_train_full, y_train_full), (X_test_full, y_test_full)=keras.datasets.cifar10.load_data()

In [6]:
X_train=X_train_full[5000:]
y_train=y_train_full[5000:]
X_valid=X_train_full[:5000]
y_valid=y_train_full[:5000]

In [7]:
# optimizer and learning rate
# Use Nadam here
optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss='sparse_categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

In [8]:
# Use early stopping
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)    # save my model
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))   # create in this path
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb] # not tensorboard_cb first

In [16]:
%tensorboard --logdir=./my_cifar10_logs --port=6006
# open tensorboard

Reusing TensorBoard on port 6006 (pid 26604), started 0:53:33 ago. (Use '!kill 26604' to kill it.)

In [17]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks = callbacks)
# try epoch 20 times first

Epoch 1/100
1407/1407 [==============================] - 26s 18ms/step - loss: 2.1352 - accuracy: 0.1813 - val_loss: 2.0627 - val_accuracy: 0.2054
Epoch 2/100
1407/1407 [==============================] - 29s 21ms/step - loss: 2.0722 - accuracy: 0.1974 - val_loss: 2.0506 - val_accuracy: 0.2110
Epoch 3/100
1407/1407 [==============================] - 29s 21ms/step - loss: 2.0329 - accuracy: 0.2086 - val_loss: 2.0575 - val_accuracy: 0.1988
Epoch 4/100
1407/1407 [==============================] - 28s 20ms/step - loss: 2.0009 - accuracy: 0.2229 - val_loss: 2.0720 - val_accuracy: 0.2182
Epoch 5/100
1407/1407 [==============================] - 27s 19ms/step - loss: 1.9966 - accuracy: 0.2202 - val_loss: 2.0502 - val_accuracy: 0.2004
Epoch 6/100
1407/1407 [==============================] - 29s 20ms/step - loss: 1.9685 - accuracy: 0.2335 - val_loss: 1.9710 - val_accuracy: 0.2224
Epoch 7/100
1407/1407 [==============================] - 28s 20ms/step - loss: 1.9437 - accuracy: 0.2434 - val_loss: 1

In [18]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)
# not really good

157/157 [==============================] - 1s 5ms/step - loss: 1.8645 - accuracy: 0.1366


[1.8645490407943726, 0.13660000264644623]

In [19]:
# try adding Batch Normalization and compare the learning curves
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [20]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
# add batchnormalization after

In [22]:
for j in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation='softmax'))    

In [23]:
# still use nadar as activation
optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [24]:
# still early stopping the differnece is that we use batchnormalization
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [25]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 45s 32ms/step - loss: 1.9873 - accuracy: 0.2812 - val_loss: 1.8169 - val_accuracy: 0.3580
Epoch 2/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.7985 - accuracy: 0.3583 - val_loss: 1.7098 - val_accuracy: 0.3898
Epoch 3/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.7136 - accuracy: 0.3895 - val_loss: 1.7063 - val_accuracy: 0.3998
Epoch 4/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.6631 - accuracy: 0.4092 - val_loss: 1.7090 - val_accuracy: 0.3892
Epoch 5/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.6182 - accuracy: 0.4242 - val_loss: 1.5813 - val_accuracy: 0.4432
Epoch 6/100
1407/1407 [==============================] - 31s 22ms/step - loss: 1.5782 - accuracy: 0.4404 - val_loss: 1.5322 - val_accuracy: 0.4464
Epoch 7/100
1407/1407 [==============================] - 30s 21ms/step - loss: 1.5408 - accuracy: 0.4522 - val_loss: 1

In [26]:
model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 3ms/step - loss: 1.3448 - accuracy: 0.1154


[1.3448237180709839, 0.11540000140666962]

In [37]:
# replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [38]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))   # SELU
model.add(keras.layers.Dense(10, activation="softmax"))

In [39]:
optimizer = keras.optimizers.Nadam(lr=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [40]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True) # save the best result
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [41]:
# make the adjustment for the input
#　normalized
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test_full - X_means) / X_stds


In [42]:
model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks)
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.9443 - accuracy: 0.3040 - val_loss: 1.8583 - val_accuracy: 0.3340
Epoch 2/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.7218 - accuracy: 0.3885 - val_loss: 1.7850 - val_accuracy: 0.3404
Epoch 3/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.6263 - accuracy: 0.4248 - val_loss: 1.6702 - val_accuracy: 0.4008
Epoch 4/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.5572 - accuracy: 0.4510 - val_loss: 1.6340 - val_accuracy: 0.4370
Epoch 5/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4963 - accuracy: 0.4760 - val_loss: 1.5613 - val_accuracy: 0.4614
Epoch 6/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4497 - accuracy: 0.4937 - val_loss: 1.5272 - val_accuracy: 0.4688
Epoch 7/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4066 - accuracy: 0.5096 - val_loss: 1.5067 

[1.4737498760223389, 0.10639999806880951]

In [43]:
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.4737 - accuracy: 0.1064: 0s - loss: 1.4708 - accuracy: 0.10


[1.4737498760223389, 0.10639999806880951]

In [44]:
# Try regularizing the model with alpha dropout. 
# Then, without retraining your model, see if you can achieve better accuracy using MC Dropout
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

In [45]:
optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


In [46]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [47]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test_full - X_means) / X_stds

In [48]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)


Epoch 1/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.8883 - accuracy: 0.3302 - val_loss: 1.7632 - val_accuracy: 0.3644
Epoch 2/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6643 - accuracy: 0.4127 - val_loss: 1.6120 - val_accuracy: 0.4234
Epoch 3/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.5732 - accuracy: 0.4486 - val_loss: 1.6175 - val_accuracy: 0.4288
Epoch 4/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.5022 - accuracy: 0.4716 - val_loss: 1.6133 - val_accuracy: 0.4586
Epoch 5/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.4453 - accuracy: 0.4940 - val_loss: 1.6009 - val_accuracy: 0.4648
Epoch 6/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.3949 - accuracy: 0.5118 - val_loss: 1.5267 - val_accuracy: 0.4866
Epoch 7/100
1407/1407 [==============================] - 12s 8ms/step - loss: 1.3487 - accuracy: 0.5327 - val_loss: 1.5520

[1.4674904346466064, 0.11540000140666962]

In [49]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [50]:
# create a new model, identical to the one we just trained (with the same weights), 
# but with MCAlphaDropout dropout layers instead of AlphaDropout layers
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) 
    else layer for layer in model.layers
])

In [51]:
# run 10 times
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)
# return the highest probability

In [52]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

0.5094